Sprint 8 - Projeto  
Parabéns! Você concluiu a seção sobre a Análise de Negócios. É hora de aplicar o conhecimento e as habilidades que você adquiriu em um projeto: um estudo de caso analítico da vida real que você concluirá por conta própria.

Quando você terminar o projeto, envie seu trabalho para o revisor do projeto para avaliação. Ele te dará feedback dentro de 48 horas. Use o feedback para fazer alterações e, em seguida, envie a nova versão de volta ao revisor do projeto.

Você poderá receber mais feedback sobre a nova versão. Isso é completamente normal. Não é incomum passar por vários ciclos de feedback e revisão.

Seu projeto será considerado concluído assim que o revisor do projeto o aprovar.

Descrição do projeto
Você se saiu muito bem no curso da TripleTen e recebeu uma oferta de estágio no departamento analítico da Y.Afisha. Sua primeira tarefa é ajudar a empresa a otimizar suas despesas com marketing.

Você tem:

Logs do servidor com dados sobre os acessos a Y.Afisha de janeiro de 2017 até dezembro de 2018
Arquivo de despejo (ou dump file, em inglês) com todos os pedidos feitos durante o período
Estatísticas de despesas com marketing
Você vai analisar:

Como as pessoas usam o produto
Quando elas começam a comprar
Quanto dinheiro cada cliente traz para a empresa
Quando as despesas serão cobertas
Instruções para completar o projeto
Passo 1. Carregue os dados e prepare-os para a análise

Armazene os dados sobre os acessos, pedidos e despesas em variáveis. Otimize os dados para a análise. Certifique-se de que cada coluna contém o tipo correto de dados.

Caminhos de arquivo:

/datasets/visits_log_us.csv. Baixe o conjunto de dados
/datasets/orders_log_us.csv. Baixe o conjunto de dados
/datasets/costs_us.csv. Baixe o conjunto de dados
Passo 2. Faça relatórios e calcule as métricas:

Produto
Quantas pessoas usam-no cada dia, semana e mês?
Quantas sessões ocorrem por dia? (um usuário pode realizar várias sessões).
Que comprimento tem cada sessão?
Com que frequência os usuários voltam?
Vendas
Quando as pessoas começam a comprar? (Na análise de KPIs, nós geralmente estamos interessados em saber o período de tempo entre o registro e a conversão - quando o usuário se torna um cliente. Por exemplo, se o registro e a primeira compra de um usuário ocorrem no mesmo dia, ele pode encaixar na categoria de Conversão 0d. Se a compra é realizada no dia seguinte, isso será a Conversão 1d. Você pode usar qualquer abordagem que permita comparar as conversões de diferentes coortes, para que você possa determinar qual coorte ou canal de marketing tem a maior eficiência)
Quantos pedidos os clientes fazem durante um determinado período de tempo?
Qual é o volume médio de uma compra?
Quanto dinheiro eles trazem para a empresa (LTV)?
Marketing
Quanto dinheiro foi gasto? No total/por origem/ao longo do tempo
Quanto custou a aquisição de clientes para cada origem?
Os investimentos valeram a pena? (ROI)
Construa gráficos para ver como essas métricas diferem para vários dispositivos e diferentes origens de anúncios e como elas mudam com o tempo.

Passo 3. Escreva uma conclusão: recomende aos especialistas de marketing quanto dinheiro e onde seria melhor investir.

Quais origens/plataformas você recomendaria? Fundamente sua escolha: em quais métricas você se concentrou? Por quê? Que conclusões você tirou ao encontrar os valores das métricas?

Formato. Complete a tarefa em um notebook Jupyter. Insira o código nas células code e explicações de texto nas células markdown. Aplique formatação e cabeçalhos.

Descrição dos dados
A tabela visits (os logs do servidor com dados sobre os acessos ao site):

Uid — identificador unívoco do usuário
Device — dispositivo do usuário
Start Ts — data e hora do início da sessão
End Ts — data e hora do final da sessão
Source Id — identificador da origem do anúncio através do qual o usuário chegou
Todas as datas nesta tabela estão no formato YYYY-MM-DD.

A tabela orders (dados sobre os pedidos):

Uid — identificador unívoco do usuário que faz um pedido
Buy Ts — data e hora do pedido
Revenue — a receita da Y.Afisha com o pedido
A tabela costs (dados sobre as despesas com marketing):

source_id — identificador da origem de anúncio
dt — data
costs — despesas com esta origem de anúncio neste dia
Como meu projeto será avaliado?
Seu projeto será avaliado com base nos seguintes critérios. Leia-os cuidadosamente antes de iniciar o projeto.

Isso é o que os revisores do projeto procuram ao avaliar seu projeto:

Como você prepara os dados para análise
Quais gráficos você fez para as métricas
Como você interpreta os gráficos resultantes
Como você calcula e interpreta cada parâmetro
Como você fundamenta suas recomendações aos especialistas de marketing e quais métricas usa
Se você segue a estrutura do projeto e mantém o código organizado
As conclusões que você tirou
Se você deixa comentários a cada passo

#**Otimização de Despesas de Marketing da Y.Afisha:**
#**Análise de Produto, Vendas e ROI**

##1. Análise Preliminar

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st

In [ ]:
# Os dataframes são carregados e os parâmetros parse_dates e dtype são utilizados para otimização de memória e tipo de dados.
costs_df = pd.read_csv('/content/drive/MyDrive/Tripleten/Data_sets/costs_us.csv', parse_dates=['dt'])
orders_df = pd.read_csv('/content/drive/MyDrive/Tripleten/Data_sets/orders_log_us.csv', parse_dates=['Buy Ts'])
visits_df = pd.read_csv('/content/drive/MyDrive/Tripleten/Data_sets/visits_log_us.csv', parse_dates=['Start Ts', 'End Ts'], dtype={'Device': 'category'})

# Renomeação das colunas para que remetam mais facilmente a seus conteúdos.
costs_df.columns = ('source_id', 'date', 'costs' )
orders_df.columns = ('buy_ts', 'revenue', 'uid')
visits_df.columns = ('device', 'end_ts', 'source_id', 'start_ts', 'uid')

###Esta etapa visa confirmar se o carregamento está correto, os tipos de dados e identificar possíveis problemas.

In [ ]:
print(costs_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   source_id  2542 non-null   int64         
 1   date       2542 non-null   datetime64[ns]
 2   costs      2542 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 59.7 KB
None


O dataframe é composto por três colunas, uma com valores 'int64', uma do tipo 'datetime64' (convertida a partir de uma coluna tipo 'str') e a terceira com valores 'float64'. A tabela contém 2542 linhas, sem valores NaN. O uso de memória é de 59.7 KB, o que indica que é leve e bem otimizado.

In [ ]:
print(orders_df.info())
print()
print(orders_df['uid'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   50415 non-null  datetime64[ns]
 1   revenue  50415 non-null  float64       
 2   uid      50415 non-null  uint64        
dtypes: datetime64[ns](1), float64(1), uint64(1)
memory usage: 1.2 MB
None

36523


Esse dataframe também é composto por três colunas, *'buy_ts'*, *'revenue'* e *'uid'*. A primeira é do tipo *'datetime64'* (convertida a partir de uma coluna tipo 'str'), a segunda contem valores *'float64'* e a terceira delas contém valores do tipo *'uint64'*, que contém números inteiros necessariamente positivos. A tabela contém 50415 linhas, sem valores NaN. O número de usuários unívocos na coluna *'uid'* é de 36523. O uso de memória é de 1.2 MB depois das melhorias feitas nos tipos de dados.


In [ ]:
print(visits_df.info())
print()
print(visits_df['uid'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   device     359400 non-null  category      
 1   end_ts     359400 non-null  datetime64[ns]
 2   source_id  359400 non-null  int64         
 3   start_ts   359400 non-null  datetime64[ns]
 4   uid        359400 non-null  uint64        
dtypes: category(1), datetime64[ns](2), int64(1), uint64(1)
memory usage: 11.3 MB
None

228169


Esse dataframe é composto por cinco colunas, 'device', 'end_ts' e 'source_id', 'start_ts' e 'uid'. A coluna 'device' foi convertida de 'str' para 'category', a segunda contem valores 'datetime64', a terceira delas contém valores do tipo 'int64', a quarta também é do tipo 'datetime64' e a quinta é do tipo 'uint64'. O df contém 359400 linhas, sem valores NaN. O número de usuários unívocos na coluna 'uid' é de 228169. O uso de memória é de 11.3 MB depois das melhorias feitas nos tipos de dados.

**É de extrema importância a otimização de memória no carregamento dos dados, especialmente para conjuntos grandes. Os dtype e parse_dates contribuem para isso.**

##2. Análise do Produto
Quantas pessoas usam-no cada dia, semana e mês?
Quantas sessões ocorrem por dia? (um usuário pode realizar várias sessões).
Que comprimento tem cada sessão?
Com que frequência os usuários voltam?